In [1]:
import cv2
import numpy as np
from matplotlib import pyplot as plt
import time
import os
import mediapipe as mp

2024-03-14 00:30:23.728711: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-03-14 00:30:23.753253: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-14 00:30:23.753282: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-14 00:30:23.753886: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-14 00:30:23.758036: I tensorflow/core/platform/cpu_feature_guar

In [2]:
mp_drawing=mp.solutions.drawing_utils
mp_holistic=mp.solutions.holistic

In [3]:
def mediapipe_detection(image,model):
    image= cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
    image.flags.writeable=False
    results=model.process(image)
    image.flags.writeable=True
    image =cv2.cvtColor(image,cv2.COLOR_RGB2BGR)
    return image,results

In [4]:
def draw_styled_landmarks(image, results):
    #face
    mp_drawing.draw_landmarks(image,results.face_landmarks,mp_holistic.FACEMESH_TESSELATION,
                              mp_drawing.DrawingSpec(color=(80,110,10),thickness=1,circle_radius=1),
                              mp_drawing.DrawingSpec(color=(80,256,121),thickness=1,circle_radius=1)
                             )
    #pose
    mp_drawing.draw_landmarks(image,results.pose_landmarks,mp_holistic.POSE_CONNECTIONS,
                              mp_drawing.DrawingSpec(color=(80,22,10),thickness=2,circle_radius=4),
                              mp_drawing.DrawingSpec(color=(80,44,121),thickness=2,circle_radius=2)
                             )
    #left hand
    mp_drawing.draw_landmarks(image,results.left_hand_landmarks,mp_holistic.HAND_CONNECTIONS,
                              mp_drawing.DrawingSpec(color=(121,22,76),thickness=2,circle_radius=4),
                              mp_drawing.DrawingSpec(color=(121,44,250),thickness=2,circle_radius=2)
                             )
    #right hand
    mp_drawing.draw_landmarks(image,results.right_hand_landmarks,mp_holistic.HAND_CONNECTIONS,
                              mp_drawing.DrawingSpec(color=(245,117,66),thickness=2,circle_radius=4),
                              mp_drawing.DrawingSpec(color=(245,66,230),thickness=2,circle_radius=2)
                             )  

In [5]:
def extract_keypoints(results):
    pose=np.array([[res.x,res.y,res.z,res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    face=np.array([[res.x,res.y,res.z] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
    lh=np.array([[res.x,res.y,res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh=np.array([[res.x,res.y,res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([pose,face,lh,rh])

In [6]:
actions=np.array(['week','monday','question'])

In [7]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard

In [8]:
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(30,1662)))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))

2024-03-14 00:30:31.142680: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-03-14 00:30:31.162476: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2256] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [9]:
model.load_weights('action.h5')

In [10]:
colors = [(245,117,16), (117,245,16), (16,117,245)]
def prob_viz(res, actions, input_frame, colors):
    output_frame = input_frame.copy()
    for num, prob in enumerate(res):
        cv2.rectangle(output_frame, (0,60+num*40), (int(prob*100), 90+num*40), colors[num], -1)
        cv2.putText(output_frame, actions[num], (0, 85+num*40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
        
    return output_frame

In [11]:
# 1. New detection variables
sequence = []
sentence = []
threshold = 0.8

cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

        # Read feed
        ret, frame = cap.read()

        # Make detections
        image, results = mediapipe_detection(frame, holistic)
        print(results)
        
        # Draw landmarks
        draw_styled_landmarks(image, results)
        
        # 2. Prediction logic
        keypoints = extract_keypoints(results)
#         sequence.insert(0,keypoints)
#         sequence = sequence[:30]
        sequence.append(keypoints)
        sequence = sequence[-30:]
        
        if len(sequence) == 30:
            res = model.predict(np.expand_dims(sequence, axis=0))[0]
            print(res)
#             print(np.argmax(res))
            print(actions[np.argmax(res)])
        #3. Viz logic
            if res[np.argmax(res)] > threshold: 
                if len(sentence) > 0: 
                    if actions[np.argmax(res)] != sentence[-1]:
                        sentence.append(actions[np.argmax(res)])
                else:
                    sentence.append(actions[np.argmax(res)])

            if len(sentence) > 5: 
                sentence = sentence[-5:]

            # Viz probabilities
            image = prob_viz(res, actions, image, colors)
            
        cv2.rectangle(image, (0,0), (640, 40), (245, 117, 16), -1)
        cv2.putText(image, ' '.join(sentence), (3,30), 
                       cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
        
        # Show to screen
        cv2.imshow('OpenCV Feed', image)

        # Break gracefully
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>


QObject::moveToThread: Current thread (0x6e5ee10) is not the object's thread (0x6cc2910).
Cannot move to target thread (0x6e5ee10)

QObject::moveToThread: Current thread (0x6e5ee10) is not the object's thread (0x6cc2910).
Cannot move to target thread (0x6e5ee10)

QObject::moveToThread: Current thread (0x6e5ee10) is not the object's thread (0x6cc2910).
Cannot move to target thread (0x6e5ee10)

QObject::moveToThread: Current thread (0x6e5ee10) is not the object's thread (0x6cc2910).
Cannot move to target thread (0x6e5ee10)

QObject::moveToThread: Current thread (0x6e5ee10) is not the object's thread (0x6cc2910).
Cannot move to target thread (0x6e5ee10)

QObject::moveToThread: Current thread (0x6e5ee10) is not the object's thread (0x6cc2910).
Cannot move to target thread (0x6e5ee10)

QObject::moveToThread: Current thread (0x6e5ee10) is not the object's thread (0x6cc2910).
Cannot move to target thread (0x6e5ee10)

QObject::moveToThread: Current thread (0x6e5ee10) is not the object's thread

<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.soluti

1/1 [==============================] - 0s 16ms/step
[1.0000000e+00 1.4900515e-24 2.3701809e-12]
week
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 14ms/step
[1.0000000e+00 1.9012003e-25 7.4533519e-14]
week
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 13ms/step
[1.000000e+00 6.490903e-26 8.143262e-15]
week
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 14ms/step
[1.00000000e+00 8.19054507e-25 1.07065685e-14]
week
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 14ms/step
[1.000000e+00 2.945731e-22 3.086107e-13]
week
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 14ms/step
[1.0000000e+00 4.1035676e-17 1.3285914e-10]
week
<class 'mediapipe.python.solution_base.SolutionOutputs'>
1/1 [==============================] - 0s 14ms/step
[1.

collect 30 frames of data

In [12]:
import cv2
import numpy as np

# Initialize webcam
cap = cv2.VideoCapture(0)

# Initialize variables
frame_count = 0
max_frames = 30
frames = []

# Loop to capture frames
while True:
    ret, frame = cap.read()
    
    if not ret:
        print("Failed to capture frame from webcam")
        break
    
    # Convert frame to grayscale (optional)
    frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    # Append frame to list
    frames.append(frame_gray)
    
    # Display frame
    cv2.imshow('Webcam Feed', frame_gray)
    
    # Increment frame count
    frame_count += 1
    
    # Break the loop when required number of frames captured
    if frame_count >= max_frames:
        break
    
    # Break the loop when 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release webcam and close OpenCV windows
cap.release()
cv2.destroyAllWindows()

# Convert frames list to numpy array
frames_array = np.array(frames)

# Print shape of numpy array
print("Shape of frames array:", frames_array.shape)


QObject::moveToThread: Current thread (0x6e5ee10) is not the object's thread (0x6cc2910).
Cannot move to target thread (0x6e5ee10)

QObject::moveToThread: Current thread (0x6e5ee10) is not the object's thread (0x6cc2910).
Cannot move to target thread (0x6e5ee10)

QObject::moveToThread: Current thread (0x6e5ee10) is not the object's thread (0x6cc2910).
Cannot move to target thread (0x6e5ee10)

QObject::moveToThread: Current thread (0x6e5ee10) is not the object's thread (0x6cc2910).
Cannot move to target thread (0x6e5ee10)

QObject::moveToThread: Current thread (0x6e5ee10) is not the object's thread (0x6cc2910).
Cannot move to target thread (0x6e5ee10)

QObject::moveToThread: Current thread (0x6e5ee10) is not the object's thread (0x6cc2910).
Cannot move to target thread (0x6e5ee10)

QObject::moveToThread: Current thread (0x6e5ee10) is not the object's thread (0x6cc2910).
Cannot move to target thread (0x6e5ee10)

QObject::moveToThread: Current thread (0x6e5ee10) is not the object's thread

Shape of frames array: (30, 480, 640)


In [13]:
frames_array

array([[[255, 255, 255, ..., 129, 130, 126],
        [255, 255, 255, ..., 129, 129, 129],
        [255, 255, 255, ..., 129, 128, 128],
        ...,
        [ 50,  50,  50, ..., 144, 132, 127],
        [ 51,  50,  49, ..., 132, 136, 136],
        [ 50,  50,  48, ...,  97,  95,  97]],

       [[255, 255, 255, ..., 129, 130, 130],
        [255, 255, 255, ..., 129, 130, 129],
        [255, 255, 255, ..., 130, 129, 129],
        ...,
        [ 50,  50,  50, ..., 144, 136, 128],
        [ 51,  50,  50, ..., 137, 144, 146],
        [ 51,  51,  50, ...,  84,  79,  94]],

       [[255, 255, 255, ..., 132, 130, 130],
        [255, 255, 255, ..., 130, 130, 130],
        [255, 255, 255, ..., 130, 130, 129],
        ...,
        [ 50,  50,  50, ..., 146, 135, 132],
        [ 50,  50,  49, ..., 134, 141, 139],
        [ 50,  50,  48, ...,  90,  85,  94]],

       ...,

       [[255, 255, 255, ..., 127, 126, 126],
        [255, 255, 255, ..., 127, 127, 126],
        [255, 255, 255, ..., 128, 127, 127